<a href="https://colab.research.google.com/github/adeviney/Advent-of-Code/blob/main/Day8_SevenSegmentSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 8: Seven Segment Search


This is my first attempt at what I thought was the hardest AoC puzzle so far. 

There is probably a more simple way to do this. Will plan to update 

## Part 1

In [1]:
def read_file(filename = 'day8input.txt'):
  with open(filename) as f:
    data = [x.strip()
    for line in f.read().split('\n')
      for x in line.split('|')]
  return data

In [2]:
def count_unique_char(str):
  return len(set(str))

In [3]:
class Entry:
    def __init__(self, signal_data):
        # ten unique signal patterns (sp)
        self.sp = [''.join(sorted(digit)) for digit in signal_data[0].split(' ')]

        #four digit output value (fdov)
        self.fdov = [''.join(sorted(digit)) for digit in signal_data[1].split(' ')]

        # the correctly wired display is shown below
        #  aaaa    
        # b    c  
        # b    c  
        #  dddd    
        # e    f  
        # e    f 
        #  gggg 
        # the following dictionary will map the correct segment wiring to the currently wired segment
        self.segment_map = {'a': '',
                            'b': '',
                            'c': '', 
                            'd': '', 
                            'e': '',
                            'f': '',
                            'g': ''}

        #7-digit display by number of unique segements
        self.display_by_num_uniq_seg = {2: [x for x in self.sp if count_unique_char(x) == 2],
                                3: [x for x in self.sp if count_unique_char(x) == 3],
                                4: [x for x in self.sp if count_unique_char(x) == 4],
                                5: [x for x in self.sp if count_unique_char(x) == 5],
                                6: [x for x in self.sp if count_unique_char(x) == 6],
                                7: [x for x in self.sp if count_unique_char(x) == 7]}



        #actual digit by number of unique segments
        self.possible_numbers = {2: [1],
                                3: [7],
                                4: [4],
                                5: [2, 3, 5],
                                6: [0, 6, 9],
                                7: [8]}

        # 7-segment display: digit
        self.final_map = {}


        self.initalize_final_mapping()
        self.get_segment_map()
        self.finish_mapping()

    def initalize_final_mapping(self):
        for display in self.sp:
            num_uniq = count_unique_char(display)
            poss_num = self.possible_numbers[num_uniq]
            if (len(poss_num) == 1):
                self.final_map[display] = poss_num[0]
            else:
                self.final_map[display] = -1

    
    def count_1478(self):
        sum = 0
        for digit in self.fdov:
            num_uniq = count_unique_char(digit)
            poss_num = self.possible_numbers[num_uniq]
            if (len(poss_num) == 1):
                sum += 1
        return sum

    def get_segment_map(self):
        #  aaaa    
        # b    c  
        # b    c  
        #  dddd    
        # e    f  
        # e    f 
        #  gggg 
        
        ### first get mapping of first segment 'a'
        # by comparing displays of 7 and 1
        # 7 will have the same integers as 1 plus one more which is 'a'
        known_segments = []

        self.segment_map['a'] = list(set(self.display_by_num_uniq_seg[3][0]) - set(self.display_by_num_uniq_seg[2][0]))[0]


        #digit 4 will contain two segments that are not present in digit 1
        # these 2 segments will be 'b' and 'd' (but we are not sure which one is which yet)
        b_d = set(self.display_by_num_uniq_seg[4][0]) - set(self.display_by_num_uniq_seg[2][0])
        
        ### digits 2,3,and 5 all have 5 unique segments
        #a,d,g are shared segments among the three digits
        # we can find all a,d,and g. since we already know what a is, remove that one 
        d_g = set(self.display_by_num_uniq_seg[5][0]).intersection(set(self.display_by_num_uniq_seg[5][1]), set(self.display_by_num_uniq_seg[5][2])) - set(self.segment_map['a'])

        #now we have (b,d) and (d,g). the similar segment between these two will be d
        self.segment_map['d'] = list(b_d.intersection(d_g))[0]

        ## we can also find b and g now!! 
        # b = (b,d) - d
        # g = (d,g) - d

        self.segment_map['b'] = list(b_d - set(self.segment_map['d']))[0]
        self.segment_map['g'] = list(d_g - set(self.segment_map['d']))[0]

        #  AAAA    
        # B    c  
        # B    c  
        #  DDDD    
        # e    f  
        # e    f 
        #  GGGG 
        #we now have a,b,d,g (shown by capital letters in the figure above)
        #5 will have the segments a,b,d,*f*,and,g
        #so we need to find a digit with 5 unique segments, containing 
        # all the segments we have found so far + 1 (this will be f)
        known_segments = [self.segment_map['a'], self.segment_map['b'],\
                         self.segment_map['d'], self.segment_map['g']]
        
        for five_segment_num in self.display_by_num_uniq_seg[5]:
            diff = set(five_segment_num) - set(known_segments)
            if (len(diff) == 1):
                self.final_map[five_segment_num] = 5
                self.segment_map['f'] = list(diff)[0]
                break
    
        known_segments += self.segment_map['f']

        #  AAAA    
        # B    c  
        # B    c  
        #  DDDD    
        # e    F  
        # e    F 
        #  GGGG 

        # we can solve for c because it is the actual value of the f segment
        # minus the other segment in the digit 1
        self.segment_map['c'] = list(set(self.display_by_num_uniq_seg[2][0]) - set(self.segment_map['f']))[0]
        known_segments += self.segment_map['c']

        # e segment is the only remaining segment that has not yet been mapped
        self.segment_map['e'] = list(set(self.display_by_num_uniq_seg[7][0]) - set(known_segments))[0]

    def finish_mapping(self):
        #remaining digits to map: 0, 2, 3, 6, 9
        DIGITS_TO_MAP = [0, 2, 3, 6, 9]
        normal_wiring = {0: 'abcefg',
                         2: 'acdeg',
                         3: 'acdfg',
                         6: 'abdefg',
                         9: 'abcdfg'}

        for digit in DIGITS_TO_MAP:
            display = normal_wiring[digit].translate(normal_wiring[digit].maketrans(self.segment_map))
            sorted_display = ''.join(sorted(display))
            self.final_map[sorted_display] = digit

    def output_value(self):
        to_return = 0
        for i, disp in enumerate(self.fdov):
            to_return += self.final_map[disp] * 10**(3-i)

        return to_return

In [4]:
raw_test_data = \
"""be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |
fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |
fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |
cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |
efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |
gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |
gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |
cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |
ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |
gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |
fgae cfgab fg bagce"""

In [5]:
# raw_test_data = \
# """
# acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
# cdfeb fcadb cdfeb cdbaf
# """

In [6]:
test_data = [x.strip()
    for line in raw_test_data.split('\n')
      for x in line.split('|') if x != '']

In [7]:
test_entries = [Entry(test_data[i:i+2]) for i in range(0, len(test_data)-1, 2)]
sum = 0
for entry in test_entries:
    four_digit_ouput = entry.output_value()
    print(four_digit_ouput)
    sum += four_digit_ouput
print("Sum", sum)

8394
9781
1197
9361
4873
8418
4548
1625
8717
4315
Sum 61229


In [8]:
data = read_file()

In [9]:
entries = [Entry(data[i:i+2]) for i in range(0, len(data)-1, 2)]

In [10]:
total = 0
for entry in entries:
  total += entry.count_1478()

In [11]:
total

349

## Part 2

In [12]:
sum = 0
for entry in entries:
    sum += entry.output_value()
print("Sum", sum)

Sum 1070957
